In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import operator
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

# Some info about what our questions are

## First we will clean the data

In [3]:
columns = ['Artist', 'Title', 'Album', '#_of_views', 'Release_date', '#_feat_artists', 
           'Artist_pop', 'Genre', 'Followers', 'Danceability', 'Track_pop', 'Explicit', 
           'Energy', 'Valence', 'Duration', 'Loudness', '#_words', 'Lyrics']
data = pd.read_csv('lyricDatasetUpdated.csv', names = columns)

Based on the greatest similarity between track titles, the data from Spotify was concatenated to the current Genius dataset. Now the dataset has additional features from Spotify. Some of the feature descriptions are grabbed straight from the Spotify API. The full list of features are as follow:<br>

1. **`Artist`** - Song artist<br>
DESCRIPTION: Name of artist
2. **`Title`** - Song title<br>
DESCRIPTION: Name of song
3. **`Album`** - Album Title<br>
DESCRIPTION: Name of album the song is from.
4. **`#_of_views`** - Number of pageviews for the lyric<br>
DESCRIPTION: Amount of views the lyric page got on the Genius website.
5. **`Release_date`** - Release date of song<br>
DESCRIPTION: The date the song was released
6. **`#_feat_artists`** - How many featured artists<br>
DESCRIPTION: The amount of artists featured on song
7. **`Artist_pop`** - Artist Popularity<br>
DESCRIPTION: How popular an artist is on Spotify. Range is 0-100.
8. **`Genre`** - Genre(s)<br>
DESCRIPTION: Genre the artist is considered as, separated by ' / '.
9. **`Followers`**<br>
DESCRIPTION: The amount of users following an artist on Spotify
10. **`Danceability`**<br>
DESCRIPTION: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. (Gotten from Spotify)
11. **`Track_pop`**<br>
DESCRIPTION: Track popularity is separate from artist popularity. It's the metric grabbed from Spotify on how popular a track is.
12. **`Explicit`**<br>
DESCRIPTION: If a song has lyrics on mature themes, or foul language, it is considered explicit. This is a binary value.
13. **`Energy`**<br>
DESCRIPTION: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
14. **`Valence`**<br>
DESCRIPTION: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
15. **`Duration`** - Duration (in Milliseconds)<br>
DESCRIPTION: The length of the song in milliseconds
16. **`Loudness`**<br>
DESCRIPTION: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
17. **`#_words`** - Total words in lyrics<br>
DESCRIPTION: How many words each lyric has.
18. **`Lyrics`**<br>
DESCRIPTION: The lyrics for the specific song.

In [4]:
data['Genre'].fillna('hip hop', inplace = True)

In [5]:
data.dropna(subset = ['Lyrics'], inplace = True)
data.reset_index(drop = True, inplace = True)

In [6]:
genres = data['Genre']
g_list = list(genres)
genre_list = ['country', 'hip hop', 'pop', 'rock', 'punk', 'metal', 
              'australian'] #Australian reggae is outlier

In [7]:
correct_guesses = {}
for index, genre in enumerate(g_list):
    if type(genre) == float:
        continue
    else:
        if len(genre) == 1:
            print(genre)
            split_genres = genre
        else:
            split_genres = genre.split(' / ') 
    for g in genre_list:
        correct = 0
        if len(split_genres) == 1 and split_genres[0] == 'rap':
            correct_guesses['hip hop'] = correct
        else:
            for split in split_genres:
                if g in split:
                    correct += 1
            correct_guesses[g] = correct
    
    correct_genre = max(correct_guesses, key=lambda key: correct_guesses[key])
    if correct_genre == 'australian':
        g_list[index] = 'pop'
    elif correct_genre == 'punk':
        g_list[index] = 'rock'
    else:
        g_list[index] = correct_genre
    correct_guesses.clear()

In [8]:
data['Genre'] = g_list

In [9]:
data['Lyrics'] = data['Lyrics'].astype(str)
data['Lyrics'] = data['Lyrics'].str.lower()
data['Lyrics'] = data['Lyrics'].str.translate(string.punctuation)
data['Lyrics'] = data['Lyrics'].str.replace('+', ' ').replace('\\', '').replace('"', '')

In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment = []
analyzer = SentimentIntensityAnalyzer()
for label, content in data['Lyrics'].items():
    sentiment.append(analyzer.polarity_scores(content))

In [11]:
neg = []
neu = []
pos = []
compound = []

In [12]:
for sentiments in sentiment:
    neg.append(sentiments['neg'])
    neu.append(sentiments['neu'])
    pos.append(sentiments['pos'])
    compound.append(sentiments['compound'])

In [13]:
data['negative_score'] = neg
data['neutral_score'] = neu
data['positive_score'] = pos
data['compound_score'] = compound

# Classification goes below here
We will conduct some experiments on which models to use, and which feature combinations work best with these models.

# Predict Popularity

In [30]:
data.head(1)

,Artist,Title,Album,#_of_views,Release_date,#_feat_artists,Artist_pop,Genre,Followers,Danceability,Track_pop,Explicit,Energy,Valence,Duration,Loudness,#_words,Lyrics
0,Miranda Lambert,Vice,The Weight of These Wings,42261,2016-07-18,0,72,country,2205152,0.568,57,False,0.562,0.247,240280,-7.775,233,sting of the needle dropping on a vinyl neon s...


In [51]:
data_pop = data.loc[:,'Danceability':'Loudness']

In [52]:
data_pop['Genre'] = data['Genre']

In [53]:
data_pop.isnull().sum()

Danceability     0
Track_pop        0
Explicit        16
Energy           0
Valence          0
Duration         0
Loudness         0
Genre            0
dtype: int64

In [54]:
data_pop.dropna(subset = ['Explicit'], inplace = True)
data_pop.reset_index(drop = True, inplace = True)

In [55]:
data_pop['Explicit'] = data_pop['Explicit'].astype(int)

In [56]:
data_pop = pd.concat([data_pop, pd.get_dummies(data_pop['Genre'], 
                    dummy_na=True)], axis = 1).drop(['Genre'], axis = 1)

In [57]:
data_pop.head(1)

,Danceability,Track_pop,Explicit,Energy,Valence,Duration,Loudness,country,hip hop,metal,pop,rock,NaN
0,0.568,57,0,0.562,0.247,240280,-7.775,1,0,0,0,0,0


In [58]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline

In [59]:
pipeline = make_pipeline(StandardScaler(), SGDRegressor())
X = data_pop.drop(['Track_pop'], axis = 1)
y = data_pop['Track_pop']

#pipeline.fit(X = X, y = y)
    
scores = cross_val_score(pipeline, X = X, y = y, cv = 5)
scores

array([-0.06318534,  0.05120728,  0.0940029 ,  0.07482506,  0.06782461])